In [2]:
import pandas as pd 
import numpy as np
from nltk.cluster import KMeansClusterer
from nltk.cluster.util import cosine_distance, euclidean_distance

import tensorflow as tf
#tf.enable_eager_execution()

import tensorflow_hub as hub


model = hub.load('../universal-sentence-encoder_4')
print ("module %s loaded" % model)
def embed(input):
    return model(input)

module <tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x7fc89c145d60> loaded


In [ ]:
extracted_n26 = pd.read_csv('extracted_n26.csv', index_col=0)
extracted_data = extracted_n26.values.tolist()
extracted_n26.head()

,market,url,title,content
0,https://support.n26.com/de-at/app-und-produkte...,de-at,Was sind Spaces?,"Setze dir Ziele, erstelle, personalisiere und ..."
1,https://support.n26.com/en-es/account-and-pers...,en-es,How to download my personal data?,The General Data Protection Regulation (GDPR) ...
2,https://support.n26.com/de-at/zahlungen-ueberw...,de-at,Wie lange dauert eine Überweisung mit Transfer...,Wenn du eine Überweisung mit TransferWise mach...
3,https://support.n26.com/en-de/app-and-features...,en-de,How to manage my visibility as an N26 user?,To grant you access to all our in-app features...
4,https://support.n26.com/en-de/payments-transfe...,en-de,How often can I withdraw cash for free?,Cash withdrawals using your Mastercard depend ...


In [ ]:
messages = extracted_n26.sample(3).answers.map(lambda x:x.split('.')[0]).values
message_embeddings = embed(messages)

for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

Message: in ios: ensure that you are using the latest version(new tab) of the n26 mobile application
Embedding size: 512
Embedding: [-0.004464928526431322, 0.0456877276301384, 0.04646744579076767, ...]

Message: it’s not possible to deactivate contactless payments
Embedding size: 512
Embedding: [-0.06443911790847778, 0.012580471113324165, 0.035532478243112564, ...]

Message: it’s important to keep your personal data up to date (for example, your phone number is necessary to connect your smartphone or to send and receive moneybeams)
Embedding size: 512
Embedding: [-0.004788025747984648, -0.03604522719979286, -0.059742219746112823, ...]



In [ ]:
def match_qas(qas_across_markets):
    # This function assigns an faq_id to the input data, thus grouping question
    # answer pairs across languages. A locale is the language of a market
    
    #Parameters:
    #    qas_across_markets (list[
    #                      (market_1,url_1,title_1,content_1),
    #                      (market_2,url_2,title_2,content_2)])
    
    
    #Returns:
    #    matched_data (list[
    #                      (faq_id_1,locale_1,market_1,title_1,content_1),
    #                      (faq_id_1,locale_2,market_2,title_2,content_2)])
    
    # set up dataframe
    qas_across_markets.columns = ['url', 'market', 'title', 'content']
    qas_across_markets['locale'] = qas_across_markets['market'].str[:2]
    qas_across_markets['faq_ids'] = [0] * len(qas_across_markets)
    qas_across_markets = qas_across_markets.drop_duplicates().dropna().reset_index()

    # create matrix with similarities
    embeddings = embed(qas_across_markets['title'])
    similarity = np.inner(embeddings, embeddings)

    # compare each sentence in english with the sentences in other languages,
    # setting group according to similarity greater than 0.4, and the greatest
    # similarity with english within each language
    languages = ['de', 'fr', 'es', 'it']
    group = 0

    for en_index, en_row in qas_across_markets[qas_across_markets['locale'] == 'en'].iterrows():
        group += 1
        qas_across_markets['faq_ids'][en_index] = group

        for lang in languages:
            lang_rows = qas_across_markets[qas_across_markets['locale'] == lang].index

            sim_tups = [(similarity[en_index, lang_index], lang_index)
                        for lang_index in lang_rows 
                          if similarity[en_index, lang_index] > 0.5]

            if len(sim_tups) > 0:
                for sim, ind in sim_tups:
                  if sim == max(sim_tups)[0]:
                      qas_across_markets['faq_ids'][ind] = group 


    matched_data = (list(qas_across_markets['faq_ids']),
                    list(qas_across_markets['locale']),
                    list(qas_across_markets['market']),
                    list(qas_across_markets['title']),
                    list(qas_across_markets['content']))
    
    return matched_data
    
final_results = match_qas(extracted_n26.copy())
%time

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


In [ ]:
df = pd.DataFrame(final_results).T
df.to_csv('final_result.csv')
df.head()

,0,1,2,3,4
0,796,de,de-at,Was sind Spaces?,"Setze dir Ziele, erstelle, personalisiere und ..."
1,1,en,en-es,How to download my personal data?,The General Data Protection Regulation (GDPR) ...
2,0,de,de-at,Wie lange dauert eine Überweisung mit Transfer...,Wenn du eine Überweisung mit TransferWise mach...
3,2,en,en-de,How to manage my visibility as an N26 user?,To grant you access to all our in-app features...
4,3,en,en-de,How often can I withdraw cash for free?,Cash withdrawals using your Mastercard depend ...


In [ ]:
df[df[0] == 769][3]

20      Comment fonctionne Google Pay ?
595          ¿Cómo funciona Google Pay?
813        Wie funktioniert Google Pay?
1309             How to use Google Pay?
1355       Wie funktioniert Google Pay?
1401          Come funziona Google Pay?
Name: 3, dtype: object

In [ ]:
df[df[0] == 828][3]

702     Comment transférer de l’argent vers des compte...
1403         How to make transfers in foreign currencies?
Name: 3, dtype: object

In [ ]:
df[df[0] == 679][3]

558      Ingresa mediante tarjeta, Apple Pay y Google Pay
1066           Ricarica con carta, Apple Pay e Google Pay
1137    Alimenter mon compte par carte, Apple Pay et G...
1159     How to top up by card, Apple Pay, and Google Pay
Name: 3, dtype: object

In [ ]:
for i, j in enumerate(df.groupby(0).count()[1]):
  if j > 2:
    print(i)

0
588
679
700
737
748
750
769
776
787
796
808
810
830


### Extras for later

In [ ]:
# Another way to do it: faster but highly dependend on embeddings

def match_qas(qas_across_markets):
    # This function assigns an faq_id to the input data, thus grouping question
    # answer pairs across languages. A locale is the language of a market
    
    #Parameters:
    #    qas_across_markets (list[
    #                      (market_1,url_1,title_1,content_1),
    #                      (market_2,url_2,title_2,content_2)])
    
    
    #Returns:
    #    matched_data (list[
    #                      (faq_id_1,locale_1,market_1,title_1,content_1),
    #                      (faq_id_1,locale_2,market_2,title_2,content_2)])
    
    # dataframe adjustments
    qas_across_markets.columns = ['url', 'market', 'title', 'content']
    qas_across_markets['locale'] = qas_across_markets['market'].str[:2]
    qas_across_markets = qas_across_markets.drop_duplicates().dropna().reset_index()

    # create matrix with similarities
    embeddings = np.inner(embed(qas_across_markets['title']), embed(qas_across_markets['title']))
    
    # compare each sentence in english with the sentences in other languages,
    # grouping them in a dict if the similarity (np.inner) is greater than 0.7
    mapping = {}
    group = 0
    for en_index, en_row in qas_across_markets[qas_across_markets['locale'] == 'en'].iterrows():
        group += 1
        mapping.setdefault(group, []).extend([en_index])
        lang_rows = qas_across_markets[qas_across_markets['locale'] != 'en'].index
        [mapping[group].append(lang_index)
          for lang_index in lang_rows
            if embeddings[en_index, lang_index] > 0.7]

    # creates a list in same order of the dataframe, to set the corresponding
    # group to each sentence
    groups = []
    for i in qas_across_markets.index:
        for k, v in mapping.items():
            noMatch = True
            if i in v:
                noMatch = False
                groups.append(k)
                break
        if noMatch:
              groups.append(0)

    qas_across_markets['faq_ids'] = groups

    matched_data = (list(qas_across_markets['faq_ids']),
                    list(qas_across_markets['locale']),
                    list(qas_across_markets['market']),
                    list(qas_across_markets['title']),
                    list(qas_across_markets['content']))
    
    return matched_data
    
final_results = match_qas(extracted_n26.copy())
%time

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


In [ ]:
## Another, but slower method

def match_qas_b(qas_across_markets):
    # This function assigns an faq_id to the input data, thus grouping question
    # answer pairs across languages. A locale is the language of a market
    
    #Parameters:
    #    qas_across_markets (list[
    #                      (market_1,url_1,title_1,content_1),
    #                      (market_2,url_2,title_2,content_2)])
    
    
    #Returns:
    #    matched_data (list[
    #                      (faq_id_1,locale_1,market_1,title_1,content_1),
    #                      (faq_id_1,locale_2,market_2,title_2,content_2)])
    
    # dataframe adjustments
    qas_across_markets.columns = ['url', 'market', 'title', 'content']
    qas_across_markets['locale'] = qas_across_markets['market'].str[:2]
    qas_across_markets = qas_across_markets.drop_duplicates().dropna()
    qas_across_markets['embedding'] = embed(qas_across_markets['title'])

    languages = ['de', 'fr', 'es', 'it']
    
    # compare each sentence in english with the sentences in other languages,
    # grouping them in a dict if the similarity (np.inner) is greater than 0.7
    mapping = {en_index:[lang_index
                        for lang in languages
                          for lang_index, lang_row in qas_across_markets[qas_across_markets['locale'] == lang].iterrows()
                            if np.inner(en_row['embedding'], lang_row['embedding']) > 0.7] + [en_index]
              
              for en_index, en_row in qas_across_markets[qas_across_markets['locale'] == 'en'].iterrows()}

    # gives a number for each group
    groups_mapping = {list(mapping.keys())[v-1]:v
                      for v in range(1, len(mapping.keys())+1)}
    
    # adds the group 0, which will represent the sentences that weren't matched in any other group
    groups_mapping[0] = 0

    # creates a list in same order of the dataframe, to set the corresponding
    # group to each sentence
    groups = []
    for i in qas_across_markets.index:
        for k, v in mapping.items():
            g = 0
            if i in v:
                g = k
                break
        groups.append(groups_mapping[g])

    qas_across_markets['faq_ids'] = groups

    matched_data = (list(qas_across_markets['faq_ids']),
                    list(qas_across_markets['locale']),
                    list(qas_across_markets['market']),
                    list(qas_across_markets['title']),
                    list(qas_across_markets['content']))
    
    return matched_data
    
    
final_results = match_qas_b(extracted_n26.copy())
%time

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.44 µs


In [ ]:
# TODO manually generate groups/subgroups, adjusting them to match kmeans labels and try to evaluate

groups = {0: ['app und produkte', 'application et produits', 'app and features', 'app e opzioni', 'application et produits', 'app y herramientas'],
          1: ['zahlungen ueberweisungen und abhebungen', 'payments transfers and withdrawals', 'paiements virements et retraits', 'pagos transferencias y retiradas', 'pagamenti prelievi e bonifici'],
          2: ['sicherheit', 'security', 'securite', 'seguridad', 'sicurezza'],
          3: ['konto und personliche informationen', 'account and personal details', 'compte et informations personnelles', 'cuenta e informacion personal', 'conto e informazioni personali'],
          4: ['mitgliedschaften und kontotypen', 'memberships and account types', 'adhesions et formules de compte', 'tipos de cuenta y suscripciones', 'conti e sottoscrizioni'],
          5: ['karten', 'cards', 'cartes', 'tarjetas', 'carte'],
          6: ['mitteilungen', 'announcements', 'annonces', 'annunci'],
          7: ['article'],
          8: ['getting started', 'come iniziare'],
          9: ['fixing an issue', 'resoudre un probleme', 'risolvi un problema']
          }

matched_data = []

qas_across_markets['groups_test'] = qas_across_markets['url'].apply(lambda x: x.split('/')[4].replace('-', ' ') if len(x.split('/')) > 4 else '')

def define_group(x):
    for k, v in groups.items():
        if x in v:
            return k

qas_across_markets['id_test'] = qas_across_markets['groups_test'].apply(lambda x: define_group(x) if x != '' else None)

qas_across_markets.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,url,market,title,content,locale,groups_test,id_test
0,https://support.n26.com/de-at/app-und-produkte...,de-at,Was sind Spaces?,"Setze dir Ziele, erstelle, personalisiere und ...",de,app und produkte,0.0
72,https://support.n26.com/de-de/app-und-produkte...,de-de,Wie kann ich meinen Kreditantrag widerrufen?,Der Widerruf von N26 Credit Darlehen ist koste...,de,app und produkte,0.0
78,https://support.n26.com/de-de/konto-und-person...,de-de,Warum funktioniert meine Video-Verifizierung n...,Einige Tipps für eine erfolgreiche Video-Verif...,de,konto und personliche informationen,3.0
67,https://support.n26.com/de-de/app-und-produkte...,de-de,Kann ich die N26 App auf mein Smartphone herun...,Die neueste Version der N26 App kannst du im A...,de,app und produkte,0.0
79,https://support.n26.com/de-at/app-und-produkte...,de-at,Shared Spaces - Vertrauenswürdiger Empfänger,Vertrauenswürdige Empfänger eines Shared Space...,de,app und produkte,0.0


In [ ]:
## CLUSTERING BY LANGUAGE

def match_qas_lang(qas_across_markets):
    # This function assigns an faq_id to the input data, thus grouping question
    # answer pairs across languages. A locale is the language of a market
    
    #Parameters:
    #    qas_across_markets (list[
    #                      (market_1,url_1,title_1,content_1),
    #                      (market_2,url_2,title_2,content_2)])
    
    
    #Returns:
    #    matched_data (list[
    #                      (faq_id_1,locale_1,market_1,title_1,content_1),
    #                      (faq_id_1,locale_2,market_2,title_2,content_2)])
    
    matched_data = []

    qas_across_markets.columns = ['url', 'market', 'title', 'content']
    qas_across_markets['locale'] = qas_across_markets['market'].str[:2]
    # added groups according to url to check clusters latter on
    qas_across_markets['groups'] = qas_across_markets['url'].apply(lambda x: x.split('/')[4].replace('-', ' ') if len(x.split('/')) > 4 else '')

    #sorting by locale, so embeddings will remain in same order
    qas_across_markets.sort_values('locale', inplace=True)

    embeddings = []

    for lang in sorted(qas_across_markets['locale'].unique()):
        temp = qas_across_markets[qas_across_markets['locale'] == lang]['title']
        embeded = list(embed(temp))
        embeddings += embeded   

    # using kmeans to cluster closest embeddings (10 clusters to represent the 10 questions segments)    
    kmeans = KMeansClusterer(10, distance=cosine_distance, repeats=20, avoid_empty_clusters=True)
    clusters = kmeans.cluster(embeddings, assign_clusters=True)
    qas_across_markets['faq_ids'] = clusters

    matched_data = (list(qas_across_markets['faq_ids']),
                    list(qas_across_markets['locale']),
                    list(qas_across_markets['market']),
                    list(qas_across_markets['title']),
                    list(qas_across_markets['content']),
                    list(qas_across_markets['groups']))
    
    return matched_data

In [ ]:
# TODO: test this code to develop a search engine

language_to_embeddings = {}

for lang in sorted(qas_across_markets['locale'].unique()):
    questions = qas_across_markets[qas_across_markets['locale'] == lang]['title']
    language_to_embeddings.setdefault(lang, []).extend(embed(questions))

embedding_dimensions = len(list(language_to_embeddings.values())[0][0])

num_index_trees = 40
language_name_to_index = {}
embedding_dimensions = len(list(language_to_embeddings.values())[0][0])

for lang in sorted(qas_across_markets['locale'].unique()):
    index = SimpleNeighbors(embedding_dimensions, metric='dot')

    for i in range(len(language_to_sentences[lang])):
        index.add_one(language_to_sentences[lang][i], language_to_embeddings[lang][i])

    index.build(n=num_index_trees)
    language_name_to_index[lang] = index


num_index_trees = 60
combined_index = SimpleNeighbors(embedding_dimensions, metric='dot')

for lang in sorted(qas_across_markets['locale'].unique()):
    for i in trange(len(language_to_sentences[language_code])):
        annotated_sentence = '({}) {}'.format(lang, language_to_sentences[lang][i])
        combined_index.add_one(annotated_sentence, language_to_embeddings[lang][i])

combined_index.build(n=num_index_trees)
